**Tiền xử lý dữ liệu**

In [2]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import re

#1. load dữ liệu
df = pd.read_csv("../data/raw/all_movies_data_1975_2025.csv")
initial_rows = df.shape[0]
print(f"Số dòng ban đầu: {initial_rows}")

# 2. chuẩn hóa tên cột
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df = df.rename(columns={"méta_score": "meta_score"})

# 3. NHÓM CỘT
money_cols = [
    "budget", "opening_weekend_gross", "grossworldwwide","gross_us_canada"
]

categorical_cols = [
    "mpa", "countries_origin", "production_company", "genres",
    "languages", "stars"
]

text_cols = [
    "description", "awards_content", "filming_locations"
]

# 4.xử lý Votes (K, M → SỐ)
def convert_votes(v):
    if pd.isna(v):
        return np.nan
    v = str(v).upper()
    if "K" in v:
        return float(v.replace("K", "")) * 1_000
    if "M" in v:
        return float(v.replace("M", "")) * 1_000_000
    try:
        return float(v)
    except:
        return np.nan

df["votes"] = df["votes"].apply(convert_votes)


# 5. XỬ LÝ CỘT DURATION (→ PHÚT)
def convert_duration_to_minutes(x):
    if pd.isna(x):
        return np.nan

    x = str(x).lower().strip()

    if x in ["unknown", "nan", ""]:
        return np.nan

    hours = 0
    minutes = 0

    h_match = re.search(r"(\d+)\s*h", x)
    m_match = re.search(r"(\d+)\s*m", x)

    if h_match:
        hours = int(h_match.group(1))
    if m_match:
        minutes = int(m_match.group(1))

    total_minutes = hours * 60 + minutes
    return total_minutes if total_minutes > 0 else np.nan

df["duration"] = df["duration"].apply(convert_duration_to_minutes)


# 6. XỬ LÝ CỘT TIỀN TỆ
for col in money_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r"[^\d.]", "", regex=True)
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")


#7. ÉP KIỂU NUMERIC
for col in ["year", "rating", "meta_score"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# 8.loại dòng không hợp lệ
df = df.dropna(subset=["year", "rating"])


# 9.điền gái trị khuyết thiếu
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

for col in text_cols:
    df[col] = df[col].fillna("No information")


# 10. LOG TRANSFORM
log_cols = [
    "votes", "budget", "opening_weekend_gross",
    "grossworldwwide", "gross_us_canada"
]

for col in log_cols:
    df[col] = df[col].clip(lower=0)
    df[f"{col}_log"] = np.log1p(df[col])

# 11. XỬ LÝ OUTLIER (WINSORIZATION)
cols_to_winsorize = [
    "votes_log", "budget_log", "opening_weekend_gross_log",
    "grossworldwwide_log", "gross_us_canada_log",
    "rating", "meta_score", "duration"
]

for col in cols_to_winsorize:
    if col in df.columns:
        df[col] = winsorize(df[col], limits=[0.01, 0.01])


# 12.kiểm tra
final_rows = df.shape[0]
print(f"\nSố dòng sau tiền xử lý: {final_rows}")
print(f"Số dòng bị loại bỏ: {initial_rows - final_rows}")

print("\nGiá trị khuyết sau xử lý (top 10):")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# lưu file
output_path = "../data/processed/clean_movies_data_1975_2025.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\nĐã lưu dữ liệu sạch tại: {output_path}")


Số dòng ban đầu: 30552

Số dòng sau tiền xử lý: 30218
Số dòng bị loại bỏ: 334

Giá trị khuyết sau xử lý (top 10):
release_date    371
title             0
duration          0
year              0
rating            0
votes             0
meta_score        0
description       0
movie_link        0
writers           0
dtype: int64

Đã lưu dữ liệu sạch tại: ../data/processed/clean_movies_data_1975_2025.csv


Tách Data - 80 train - 20 test

In [3]:
from sklearn.model_selection import train_test_split

df_clean = pd.read_csv("../data/processed/clean_movies_data_1975_2025.csv")

print(f"\nShape dataset sạch: {df_clean.shape}")

train_df, test_df = train_test_split(
    df_clean,
    test_size=0.2,
    random_state=42,
    stratify=None  
)

print(f"Train shape: {train_df.shape}")
print(f"Test shape : {test_df.shape}")

# Lưu file train/test
train_path = "../data/split/train.csv"
test_path  = "../data/split/test.csv"

train_df.to_csv(train_path, index=False, encoding="utf-8-sig")
test_df.to_csv(test_path, index=False, encoding="utf-8-sig")

print("\nSplit data hoàn tất!")
print(f"Train lưu tại: {train_path}")
print(f"Test  lưu tại: {test_path}")


Shape dataset sạch: (30218, 28)
Train shape: (24174, 28)
Test shape : (6044, 28)

Split data hoàn tất!
Train lưu tại: ../data/split/train.csv
Test  lưu tại: ../data/split/test.csv
